In [ ]:
!pip install --upgrade --user google-cloud-aiplatform google-cloud-storage 'google-cloud-bigquery[pandas]'

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
# get project ID
PROJECT_ID = ! gcloud config get project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"
if PROJECT_ID == "(unset)":
    print(f"Please set the project ID manually below")

In [2]:
# define project information
if PROJECT_ID == "(unset)":
    PROJECT_ID = "dark-caldron-414803"  # @param {type:"string"}

# generate an unique id for this session
from datetime import datetime

UID = datetime.now().strftime("%m%d%H%M")

In [3]:
import sys

# if it's Colab runtime, authenticate the user with Google Cloud
if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [5]:
# load the BQ Table into a Pandas Dataframe
import pandas as pd
from google.cloud import bigquery

QUESTIONS_SIZE = 1000

bq_client = bigquery.Client(project=PROJECT_ID)


QUERY_TEMPLATE2="""
select a.quotation_No,a.room_type,a.service, b.duration from `final_dataset.project_schedule` as a
        inner join `final_dataset.quotation_duration` as b
        ON a.quotation_No = b.quotation and a.room_type = b.room
        group by a.quotation_No,a.room_type,a.service,b.duration
"""

query2 = QUERY_TEMPLATE2.format(limit=QUESTIONS_SIZE)
query_job2 = bq_client.query(query2)
rows2 = query_job2.result()
df2 = rows2.to_dataframe()

QUERY_TEMPLATE = """ Select quotation_id as id,service_types from final_dataset.quotation_values
        """
query = QUERY_TEMPLATE.format(limit=QUESTIONS_SIZE)
query_job = bq_client.query(query)
rows = query_job.result()
df = rows.to_dataframe()

# examine the data
df.head()
df2.head()

,quotation_No,room_type,service,duration
0,Q0008,Lobby/passages,Furniture,27
1,Q0008,Living Room,Furniture,42
2,Q0008,Kids Room,Furniture,40
3,Q0008,Bedroom,Furniture,50
4,Q0008,Pooja Room,Furniture,27


In [6]:
# init the vertexai package
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
# init the aiplatform package
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)



In [8]:
# Load the text embeddings model
from vertexai.preview.language_models import TextEmbeddingModel

model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

In [9]:
import time
import tqdm  # to show a progress bar

# get embeddings for a list of texts
BATCH_SIZE = 5


def get_embeddings_wrapper(texts):
    embs = []
    for i in tqdm.tqdm(range(0, len(texts), BATCH_SIZE)):
        time.sleep(1)  # to avoid the quota error
        result = model.get_embeddings(texts[i : i + BATCH_SIZE])
        embs = embs + [e.values for e in result]
    return embs

In [18]:
test_embeddings = get_embeddings_wrapper(["Describe a project schedule for work in pooja room, bed room and living room"])

100%|██████████| 1/1 [00:02<00:00,  2.42s/it]


In [10]:
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint("98203980546441216")

In [14]:
DEPLOYED_INDEX_ID = "project_schedule_final_03020808"

In [20]:
# Test query
response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=test_embeddings,
    num_neighbors=5,
)
print(response)
# show the result
import numpy as np

id_arr=[]

for idx, neighbor in enumerate(response[0]):
    id = np.str_(neighbor.id)
    similar = df.query("id == @id", engine="python")
    id_arr.append(id)
    print(f"{similar.id.values[0]} {similar.service_types.values[0]}");
print('-------------------------- ')
for id in id_arr:
  similar = df2.query('quotation_No == "%s"' %id, engine="python")
  for i in range(0,len(similar)):
    print(f"{similar.quotation_No.values[i]} {similar.service.values[i]} {similar.room_type.values[i]} {similar.duration.values[i]}")
  # print(f"{similar.quotation_No.values[0]} {similar.service.values[0]} {similar.room_type.values[0]} {similar.duration.values[0]}")

[[MatchNeighbor(id='Q0012', distance=0.7434127926826477, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[]), MatchNeighbor(id='Q0020', distance=0.7402687072753906, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[]), MatchNeighbor(id='Q0036', distance=0.7349977493286133, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[]), MatchNeighbor(id='Q0002', distance=0.7304138541221619, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[]), MatchNeighbor(id='Q0028', distance=0.7304138541221619, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[])]]
Q0012 Bedroom, Kitchen, Living Room, Lobby/passages, Pooja Room
Q0020 Bedroom, Living Room, Lobby/passages, Pooja Room, Study Room
Q0036 Bedroom, Entertainment Room, Kitchen, Living Room, Lobby/passages, Pooja Room
Q0002 Bedroom, Kitchen, Living Room, Lobby/passages, Pooja Room, Washroom
Q0028 Bedroom, Kitchen, Living Room, Lobby/passages, Pooja Room

In [ ]:
import sys

# if it's Colab runtime, authenticate the user with Google Cloud
if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

import pandas as pd
from vertexai.language_models import TextGenerationModel

generation_model = TextGenerationModel.from_pretrained("text-bison@001")

In [ ]:
context = """
You are a Project Scheduling Manager and you have to provide a schedule of completing the provided details.
You will be provided Service Type, Tier, Room Type and location which is defined by latitudes and longitudes.
Service Type can be of any type which is listed below wooden flooring , furniture, bathroom , wooden ceiling
,civil, electrical, Doors and Windows , Miscellanious , Demoilition , Paints, PEST, Fitted Furniture ,
Modular Kitchen, General .
Site can belong to Tier 1, Tier 2 , Tier 3.
A Tier 1 site is larger in size and has more number of rooms.
A Tier 2 site is medium in size and has medium number of rooms.
A Tier 3 site is small in size and has less number of rooms.
and Room Type can be of Kids Room , Entertainment Room , Washroom , Terrace , Pooja Room,
Kitchen , Lawn , Porch, Study room , Bedroom, Looby/Passage , Other Room.

The input text will be in sequece of Room Type, list of services required the room type and duration.
Sample Output text will be:
Type: Pooja Room
SubTask: Furniture Delivery
Duration: 5 days

Type: Pooja Room
SubTask: furniture Installation
Duration: 5 days

"""

question= f"Create a project Schedule for a site belonging to location {similar.Location.values[0]} {similar.Tier.values[0]} {similar.Room_Type.values[0]}  {similar.Service.values[0]},  {similar.duration.values[0]} days"
prompt = f"""Answer the question given the context below as {{Context:}}. \n
If the answer is not available in the {{Context:}} and you are not confident about the output,
please say "Information not available in provided context". \n\n
Context: {context}?\n
Question: {question} \n
Answer:
"""

# print("[Prompt]")
# print(prompt)

print("[Response]")
print(
     generation_model.predict(
        prompt,
    ).text
)

[Response]
Type: Pooja Room
SubTask: Furniture Delivery
Duration: 5 days

Type: Pooja Room
SubTask: furniture Installation
Duration: 5 days

Type: Pooja Room
SubTask: Furniture Assembly
Duration: 5 days

Type: Pooja Room
SubTask: Furniture Final Inspection
Duration: 2 days
